## 特徴量選択
- 特徴量選択は大きく分けて以下の２通りの方法で選択しているようである。
  - 相関係数などの統計量から上位のものを選択
  - 変数重要度（ランダムフォレストなど）から上位のものを選択

In [1]:
import numpy as np
import pandas as pd

### データの読み込み

In [2]:
train = pd.read_csv('../../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']

## 相関係数
- 目的変数との相関係数を計算し、上位5位を表示

In [3]:
corrs = []
for c in train_x.columns:
    corr = np.corrcoef(train_x[c], train_y)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)

idx = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
dict(zip(top_cols, top_importances))

{'medical_info_a1': 0.21805214074801646,
 'medical_keyword_5': 0.21368557020355813,
 'medical_keyword_4': 0.18109642414513719,
 'medical_keyword_3': 0.16723960583428768,
 'age': 0.1515530760883347}

## スピアマンの順位相関係数
- 元の値の順位の着目し、相関係数を計算（参考程度に）

In [4]:
import scipy.stats as st

In [5]:
corrs_sp = []
for c in train_x.columns:
    corr_sp = st.spearmanr(train_x[c], train_y).correlation
    corrs_sp.append(corr_sp)
corrs_sp = np.array(corrs_sp)

idx = np.argsort(np.abs(corrs_sp))[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs_sp[idx][:5]
dict(zip(top_cols, top_importances))

{'medical_info_a1': 0.22182331060138086,
 'medical_keyword_5': 0.21368557020355833,
 'medical_keyword_4': 0.18109642414513724,
 'medical_keyword_3': 0.16723960583428796,
 'age': 0.15170291108416198}

⇒ （ピアソンの）相関係数と重要度ランキングtop5は変わらないようである。

## カイ２乗統計量
- 特徴量が非負の値で分類タスクである必要がある。
- カイ２乗分布が独立性の検定に使用されることから、これを変数の重要度として利用することもあるようだ。

In [6]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [7]:
x = MinMaxScaler().fit_transform(train_x)
c2, _ = chi2(x, train_y)

idx = np.argsort(c2)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
dict(zip(top_cols, top_importances))

{'medical_keyword_5': 0.21368557020355813,
 'medical_keyword_4': 0.18109642414513719,
 'medical_keyword_3': 0.16723960583428768,
 'product_9': 0.11706114566211498,
 'medical_keyword_2': 0.11846089927105277}

⇒ 順位が変わった。

## 相互情報量
- 変数が従属の場合に１に近づき、独立の場合は０になる統計量である。

In [8]:
from sklearn.feature_selection import mutual_info_classif

In [9]:
mi = mutual_info_classif(train_x, train_y)

idx = np.argsort(mi)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
dict(zip(top_cols, top_importances))

{'medical_info_a1': 0.21805214074801646,
 'weight': 0.004378080391541526,
 'age': 0.1515530760883347,
 'medical_keyword_5': 0.21368557020355813,
 'medical_keyword_4': 0.18109642414513719}

## Random Forest 特徴量の重要度

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
clf = RandomForestClassifier(n_estimators=10, random_state=71)
clf.fit(train_x, train_y)
fi = clf.feature_importances_

idx = np.argsort(fi)[::-1]
top_cols, top_importances = train_x.columns.values[idx][:5], corrs[idx][:5]
dict(zip(top_cols, top_importances))

{'medical_info_a1': 0.21805214074801646,
 'weight': 0.004378080391541526,
 'age': 0.1515530760883347,
 'medical_info_a2': -0.09160657096027597,
 'height': -0.015877737491943444}

## XGBoost 特徴量の重要度
- xgboostでは以下の特徴量重要度を出力できる。
  - ゲイン
    - その特徴量の分岐により得た目的関数の減少
  - カバー
    - その特徴量の分岐により分岐させられたデータの数
  - ウェイト
    - その特徴量が分岐に現れた数

get_scoreのimportance_type引数に設定する。デフォルトがweightなので、total_gainにする。

In [12]:
import xgboost as xgb

In [13]:
dtrain = xgb.DMatrix(train_x, label=train_y)
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50
model = xgb.train(params, dtrain, num_round)

In [14]:
fscore = model.get_score(importance_type='total_gain')
fscore = sorted([(k, v) for k, v in fscore.items()], key=lambda x: x[1], reverse=True)
print('xgboost importance')
print(fscore[:5])

xgboost importance
[('weight', 2650.4720100040004), ('medical_info_a1', 2217.825263643517), ('height', 1846.56528821), ('age', 1450.881469502299), ('medical_info_a2', 1139.6175488040003)]
